In [ ]:
# run the OCR outputs against a coherence detection model
    # create markov chains of the letter transitions
    # create frequency distributions of letter transitions in English
    # score the OCR strings based on how many 'common' or 'real' letter transitions they have
    # the highest scoring string is the most likely to have originated from the properly oriented image
# use the string with the highest score